# Análisis de empleabilidad 

In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import _plotly_utils.basevalidators
import plotly.express as px
from plotly.figure_factory import create_table
import plotly.figure_factory as ff
import plotly.offline as po
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [141]:
def stacked_bar_width_plot(df, value_cols, width_col, colors=None, **subplot):
    
    categories = df.index.to_list()
    width = df[width_col]
    htmpl = width_col + ': %{customdata}, %{y}'
    x = np.cumsum([0] + list(width[:-1]))
    colors = colors or [None, ] * len(value_cols)

    figure = subplot.pop('figure', go.Figure())
    for colname, color in zip(value_cols, colors):
        figure.add_trace(go.Bar(name=colname, x=x, y=df[colname], width=width, hovertemplate=htmpl, customdata=width, offset=0, marker_color=color), **subplot)
        figure.update_layout(title='Análisis de desempeño académico y profesional, en Chile y en el Extranjero, encuestados Graduados 2012-2021',
                             font=dict(
        family="Rockwell", size=12),
        legend=dict(
        x=1,
        y=1,
        traceorder="reversed",
        font=dict(
            family="Rockwell",
            size=14,
            color="black"
        ),
        bgcolor="white",
        bordercolor="white",
        borderwidth=2
    )
)
    return figure.update_xaxes(
        tickvals=x + np.array(width) / 2,
        ticktext=categories,
        range=[0, np.sum(width)], **subplot
    ) \
        .update_yaxes(tickformat=',.0%', range=[0, 1], row=subplot.get('row'), col=subplot.get('col')) \
        .update_layout(barmode='stack', bargap=0)


def mekko_plot(df, unit_name=None, colors=None, **subplot):
    
    value_cols = df.columns
    w = pd.DataFrame({unit_name: df.sum(axis='columns')})
    w[value_cols] = df.div(w[unit_name], axis='index')

    return stacked_bar_width_plot(w, value_cols, width_col=unit_name, colors=colors, **subplot)

## Distribución de empleo

In [142]:
path_file_perfil    = 'egresados_sorted_perfil.xlsx'
df_egresados_perfil = pd.read_excel(path_file_perfil, sheet_name='egresados_perfil', index_col=0)

In [143]:
fields_perfil = ['mail', 'Academia', 'Profesional', 'País']
df_egresados_subset_columns_perfil = pd.read_excel(path_file_perfil, index_col=0, usecols=fields_perfil)

In [144]:
path_file    = 'egresados_sorted.xlsx'
df_egresados = pd.read_excel(path_file, sheet_name='egresados', index_col=0)

In [145]:
fields = ['mail', 'Empleo', 'Estatus_empleo', 'País']
df_egresados_subset_columns = pd.read_excel(path_file, index_col=0, usecols=fields)

## Tabla de datos

In [146]:
large_rockwell_template = dict(
    layout=go.Layout(title_font=dict(family="Rockwell", size=16))
)

fig = go.Figure(data=[go.Table(
    header=dict(values=list('<b>' + df_egresados_subset_columns.columns + '</b>'),
    fill_color='darkblue', 
    align='left', font=dict(
        family="Rockwell", color='white', size=12), height=20),
    cells=dict(values=[df_egresados.Empleo, df_egresados.Estatus_empleo, df_egresados.País],
    fill_color='lavender',
    align='left', font=dict(
        family="Rockwell", size=12), height=30))
    ])
fig.update_layout(width=800, height=1000)    

fig.update_layout(title="Tabla. Datos generales de empleo encuestados Graduados 2012-2021",
                  template=large_rockwell_template)

fig.show()

## ¿En cuáles países actualmente trabajan?

In [147]:
loc_df = pd.read_csv("location_map.csv")
new_df = pd.merge(df_egresados_subset_columns[["País","Empleo"]], loc_df, left_on="País", right_on="COUNTRY")
new_df = new_df.groupby(["País", "CODE"])["Empleo"].aggregate('count').reset_index()
new_df.columns = ["País", "Code", "Count"]

data = [ dict(
        type = 'choropleth',
        locations = new_df['Code'],
        z = new_df['Count'],
        text = new_df['País'],
        colorscale = [[0,"rgb(210, 210, 210)"], [4500,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Cuenta'),
      ) ]

layout = dict(
    title = 'Países en los que actualmente trabajan encuestados Graduados 2012-2021', font=dict(
        family="Rockwell", size=16),
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )

## ¿En qué Instituciones desarrollan actualmente su profesión?

In [148]:
platform=df_egresados.Empleo.dropna()
fig=px.histogram(data,x=platform,color=platform,width=1200, height=600,template='ggplot2',orientation='v')
fig.update_layout(title='Desarrollo profesional encuestados Graduados 2012-2021', font=dict(
        family="Rockwell", size=12),
                 xaxis_title='Lugares de desarrollo profesional')
fig.show()

## ¿Cuál es el estatus de empleo?

In [149]:
temp_series = df_egresados_subset_columns['Estatus_empleo'].value_counts()
labels = (np.array(temp_series.index))
sizes = (np.array((temp_series / temp_series.sum())*100))

trace = go.Pie(labels=labels, values=sizes, marker=dict(line=dict(color='#797D7F', width=1.5)), hole=.4)
layout = go.Layout(
    title='Distribución de estatus de empleabilidad de encuestados Graduados 2012-2021',
    title_font_family="Rockwell", 
    annotations=[dict(text='Estatus', x=0.5, y=0.5, font_size=12, showarrow=False)],
    font=dict(
        family="Rockwell", size=12)
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="gender")

## ¿Cuál es la distribución de empleos por países?

In [150]:
fig=px.histogram(df_egresados_subset_columns,x=df_egresados_subset_columns.Empleo, color=df_egresados.País, template='ggplot2', orientation='v')
fig.update_layout(
    title="Distribución empleos por países encuestados Graduados 2012-2021", yaxis_title='Cuenta', font=dict(
        family="Rockwell", size=11),
    xaxis_title="")
fig.update_xaxes(showline=True, linewidth=3, linecolor='lightgrey')
fig.update_yaxes(showline=True, linewidth=3, linecolor='lightgrey')
fig.show()

## ¿Cómo se distribuye el desempeño académico y profesional?

In [151]:
df = pd.DataFrame(dict(Perfil=['Desempeño académico','Desempeño profesional'],
                       Chile = [4, 12],
                       Extranjero = [6, 2])).set_index('Perfil')

mekko_plot(df, unit_name='some_label', colors=['LightSteelBlue', '#636EFA'])